In [1]:
# required for the skeletonizing algorithm
# !pip install opencv-contrib-python

In [2]:
#important import libraries
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import cv2 as cv
# from google.colab.patches import cv2_imshow
from IPython.display import clear_output
# from google.colab import files


In [3]:
#list the filenames in the folder containing your dataset
filenames=os.listdir('Total Images')
#strip the file extension from the file names
filenames_split=[filename.replace('.tif', '') for filename in filenames]

In [4]:
def directionalHistogram(img, direction='H'):
  #function to compute the intensity histograms of an image in x and y directions
    (w,h) = img.shape
    sum = []
    pixel_count=0
    if(direction=='H'):
        for j in range(w-1):
          for i in range(h-1):
            pixel=img[j,i]
            if(pixel==255):
              pixel_count+=1
          sum.append(pixel_count)
          pixel_count=0
    else:
       for j in range(h-1):
          for i in range(w-1):
            pixel=img[i,j]
            if(pixel==255):
              pixel_count+=1
          sum.append(pixel_count)
          pixel_count=0
    return sum

def crop_image(image, direction='H'):
  #function to crop the image in vertical and horizontal directions
  w,h=image.shape
  if(w<10 or h<10):
    return image

  hist=directionalHistogram(image, direction )
  flipped_hist=np.flip(hist)
  startpos=0
  endpos=0
  for i in range(1, len(hist)-1):
    if(hist[i-1]==0 and hist[i]==0 and hist[i+1]!=0):
      startpos=i
      break
  for i in range(1, len(flipped_hist)-1):
    if(hist[i-1]==0 and hist[i]==0 and hist[i+1]!=0):
      endpos=len(flipped_hist)-1-i
      break

  diff=np.abs(startpos-endpos)
  if(direction=='H' and diff>=10 and startpos<endpos):
    if(startpos<endpos):
      return image[startpos:endpos,:]
    else:
      return image[endpos:startpos,:]
  elif(direction=='V' and diff>=10 and startpos<endpos):
    if(startpos<endpos):
      return image[:,startpos:endpos]
    else:
      return image[:,endpos:startpos]
  else:
    return image

image_width = 64
image_height = 32
def distortion_free_resize(image, img_size=(image_height, image_width)):
  #resizing the images without distortion using tensorflow
    h,w= img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

    # Check tha amount of padding needed to be done.
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    # Only necessary if you want to do same amount of padding on both sides.
    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height // 2

    if pad_width % 2 != 0:
        width = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
            [pad_height_top, pad_height_bottom],
            [pad_width_left, pad_width_right],
            [0, 0],
        ],
    )

    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image

In [5]:
!mkdir processed_images
for m in range(len(filenames)):
  # read the image in grey scale
  image = cv.imread('Total Images/'+filenames[m],0)
  # skeletonize the image
  #image=cv.ximgproc.thinning(image)
  #crop the image horizontally
  image_v_cropped=crop_image(image, 'V')
  #convert image to RGB
  image_v_cropped=cv.cvtColor(image_v_cropped,cv.COLOR_GRAY2RGB)
  #distortionless resize with tensorflow
  image=distortion_free_resize(image_v_cropped)
  #convert image back to np array as grey scale and save it as jpg
  image=cv.cvtColor(image.numpy(),cv.COLOR_RGB2GRAY)
  image=np.rot90(image)
  cv.imwrite("processed_images/"+filenames_split[m]+".jpg", image)

# !zip -r /content/processed_images.zip /content/processed_images
# files.download("/content/processed_images.zip")











